In [1]:
'''
import theano
import theano.tensor as T
import theano.tensor.nnet as conv
import numpy as np
'''
import cv2

In [2]:
cap = cv2.VideoCapture("http://10.0.0.5:8080/video?x.mjpeg")

In [3]:
# In[83]:

## A randomly initialized convolutional layer with 2 feature maps
# Input : (RGB) 3x639x516
# Receptive Field size : 9x9

# Random element
rng = np.random.RandomState(23455)

# Input 
input = T.tensor4('input')

# weight shape, w [depth_of_layer_m,depth_of_layer_m-1,filter_height,filter_width]
wshape = (2,3,9,9)

# bound on w values
wbound = np.sqrt(3 * 9 * 9)

# weight initialize with random numbers
wval = np.asarray(rng.uniform(low=-1/wbound,high=1/wbound,size=wshape),dtype=input.dtype)

# build shared variable w
w = theano.shared(wval,name='w')


# In[84]:

# setup bias
b = theano.shared(np.asarray(rng.uniform(-0.5,0.5,(2,)),dtype=input.dtype),name='b')


In [5]:
# build symbolic expression for convolution layer
conv_out = conv.conv2d(input,w)


# In[86]:

# associate the bias term with the output of conv_layer
output = T.nnet.sigmoid(conv_out + b.dimshuffle('x', 0, 'x', 'x'))


# In[87]:

# compile function
convf = theano.function([input],output)

In [13]:
# kernel for sharpening image
#Create the identity filter, but with the 1 shifted to the right!
kernel = np.zeros( (8,8), np.float32)
kernel[4,4] = 2.0   #Identity, times two! 

#Create a box filter:
boxFilter = np.ones( (12,12), np.float32) / 81.0

#Subtract the two:
kernel = kernel - boxFilter

In [3]:
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if frame.shape[0] > 0 and frame.shape[1] > 0:
        resized = cv2.resize(frame,(480,270), interpolation = cv2.INTER_CUBIC)
        # sharpen image
        #resized = cv2.filter2D(resized, -1, kernel)
        cv2.imshow('frame',resized)
        #img = np.asarray(resized, dtype=theano.config.floatX)/256.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        #img_ = img.transpose(2,0,1).reshape(1,3,480,270)
        #omg = convf(img_)
        #cv2.imshow('f01',omg[0,0,:,:])
        #cv2.imshow('f02',omg[0,1,:,:])